In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

print("API key loaded")

API key loaded


In [10]:
from dotenv import load_dotenv

In [11]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
#from langchain_community.vectorstores import FAISS
#from langchain_community.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [12]:
from langchain_openai import ChatOpenAI


In [13]:
pip install langchain-community


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader
#from langchain_community.document_loaders.markdown import UnstructuredMarkdownLoader  # Good for .md

# List to hold all loaded documents
docs = []

# Load PDFs
pdf_loader = DirectoryLoader(
    "documents/",
    glob="**/*.pdf",          # Only PDF files
    loader_cls=PyPDFLoader,
    show_progress=True
)
docs.extend(pdf_loader.load())

# # Load TXT files
# txt_loader = DirectoryLoader(
#     "documents/",
#     glob="**/*.txt",          # Only TXT files
#     loader_cls=TextLoader,
#     show_progress=True
# )
# docs.extend(txt_loader.load())

# # Load Markdown files
# # md_loader = DirectoryLoader(
# #     "documents/",
# #     glob="**/*.md",           # Only MD files
# #     loader_cls=UnstructuredMarkdownLoader,  # Handles .md well
# #     show_progress=True
# # )
# # docs.extend(md_loader.load())

# # Optional: Add more types, e.g., .docx with UnstructuredWordDocumentLoader, etc.
print(f"Total loaded documents: {len(docs)}")
# print(f"First document: {docs[0].page_content}")

[print(f"Document {i+1}: {doc.page_content}") for i, doc in enumerate(docs[:3])]


100%|██████████| 1/1 [00:00<00:00,  5.99it/s]

Total loaded documents: 3
Document 1: 1 / 3  
CV  
 
 
Impressive communication skills, keen interest to learn, good interpersonal skills, ability to 
adapt to changes, good leadership qualities and a team player. Excellent writing skill with 
ability to plan and formulate strategies for effective results; Effective Verbal communication 
and ability to work under pressure to meet deadlines. A skillful armed trained senior officer.    
EDUCATION   
SCHOOLS ATTENDED WITH DATES  
 
 Nigeria Communications Commission (NCC)     2025-2026 
 Nigeria Defence Intelligence College (DIA)          2024  
 Registered as an Engineer with COREN          2019  
 Member of Nigerian Society of Engineers          2018  
 Member of Nigeria Institute of Elect & Elect. Engineers      2018  
 University of Buea, Cameroon              2016  
 National Open University of Nigeria            2014  
 National Information technology Development Agency      2010  
 Olabisi Onabanjo University, Ago-Iwoye, O

[None, None, None]

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Experiment: Start with chunk_size=1000, overlap=200 (overlap helps connect ideas)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Characters, not tokens—use tiktoken for precision if needed
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", "•", "-", " "]

)

chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks.")

# Test different sizes: Try chunk_size=500 and see if retrieval improves later.
# Split documents
#chunks = text_splitter.split_documents(docs)

print(f"Split {len(docs)} documents into {len(chunks)} chunks")
for i, chunk in enumerate(chunks[:22]):
    print(f"\nChunk {i+1}: {chunk.page_content}")
# for i, chunk in enumerate(chunks[9:12], start=1):
#     print(f"\nChunk {i}: {chunk.page_content}")


Created 8 chunks.
Split 3 documents into 8 chunks

Chunk 1: 1 / 3  
CV  
 
 
Impressive communication skills, keen interest to learn, good interpersonal skills, ability to 
adapt to changes, good leadership qualities and a team player. Excellent writing skill with 
ability to plan and formulate strategies for effective results; Effective Verbal communication 
and ability to work under pressure to meet deadlines. A skillful armed trained senior officer.    
EDUCATION   
SCHOOLS ATTENDED WITH DATES  
 
 Nigeria Communications Commission (NCC)     2025-2026 
 Nigeria Defence Intelligence College (DIA)          2024  
 Registered as an Engineer with COREN          2019  
 Member of Nigerian Society of Engineers          2018  
 Member of Nigeria Institute of Elect & Elect. Engineers      2018  
 University of Buea, Cameroon              2016

Chunk 2:  University of Buea, Cameroon              2016  
 National Open University of Nigeria            2014  
 National Information tech

In [17]:
# import os

# pdf_folder = "documents"

# pdf_files = sorted([
#     os.path.join(pdf_folder, f)
#     for f in os.listdir(pdf_folder)
#     if f.endswith(".pdf")
# ])

# print("PDF files found:")
# for i, f in enumerate(pdf_files, 1):
#     print(f"{i}. {f}")


In [18]:
# from langchain_community.document_loaders import PyPDFLoader

# fourth_pdf_path = pdf_files[3]  # 5th PDF (0-based index)
# loader = PyPDFLoader(fourth_pdf_path)
# docs = loader.load()


In [19]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=210,  # Characters, not tokens—use tiktoken for precision if needed
#     chunk_overlap=20,
#     length_function=len,
#     separators=["\n\n", "\n", "•", "-", " "]

# )

# chunks = text_splitter.split_documents(docs)
# print(f"Pages in 4th PDF: {len(docs)}")
# print(f"Chunks created: {len(chunks)}")

# for i, chunk in enumerate(chunks[:3], 1):
#     print(f"\nChunk {i}:\n{chunk.page_content[:300]}")



### Embeddings

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=api_key
    
)
# Test embedding
test_embedding = embeddings.embed_query("What is the title of thesis that sherif wrote during his Msc?")
print(f"Embedding dimension: {len(test_embedding)}")
print(f"First 5 values: {test_embedding[:5]}")

Embedding dimension: 1536
First 5 values: [0.019431181252002716, 0.019020289182662964, -0.013691957108676434, 0.012154429219663143, -0.051480699330568314]


In [21]:
pip 


Usage:   
  c:\Users\DELL\OneDrive\Desktop\RAG\rag_env\Scripts\python.exe -m pip <command> [options]

Commands:
  install                     Install packages.
  lock                        Generate a lock file.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  inspect                     Inspect the python environment.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requ

### Vector Store

In [22]:
from langchain_chroma import Chroma

# Create vector store from documents
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_db"
)

# To load later: vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

print(f"Vector store created with {len(chunks)} chunks")

# Test similarity search
query = "Thesis that sherif wrote during his Bsc?"
results = vectorstore.similarity_search(query, k=2)

print(f"\nQuery: {query}")
for i, doc in enumerate(results):
    print(f"\nResult {i+1}: {doc.page_content}")

Vector store created with 8 chunks

Query: Thesis that sherif wrote during his Bsc?

Result 1: 2 / 3  
 Master In information Technology          2014  
 Certificate of Proficiency in IT (A+)          2010  
 B.Sc. (Hons) Computer Engineering          2008  
 National Diploma (ND) Computer Science        2002   
EXPERIENCE  
 
 National Committee Member TC111&TC21(SON)       2022  
 Nigeria Security and Civil Defence Corps         2012 Till Date  
 Post: HOU ICT/Mobile CCTV Engineer (2019 Till Date)  Radio/signal com. Instructor/operator.  
 Team leader enforcement of compliance on Standard & Regulations.  
 Dezeem Automobile Ibadan              2012             
 London Capital Ltd. Lagos Nigeria.            2011  
 Post: Marketing Consultant/IT Specialist  
 INEC Ogun State                  2010 – 2011   
o Post: Ad-Hoc Staff

Result 2: 3 / 3  
 United Nations Police  Operational-Logistics-Supports  International Humanitarians Law.  
 Human’s Right and Peacekeeping  

## Building RAG with LCEL (LangChain Expression Language)

### Simple RAG Chain

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Create LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=api_key
)

# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Cosine similarity
    search_kwargs={"k": 3}  # Return top 3 chunks
)
# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant for my personal documents. Answer using ONLY the provided context.If you don't know, say so. Always cite sources"),
    ("human", "{question}\n\nContext:\n{context}")
])

# Helper function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build RAG chain using LCEL
rag_chain = (
    RunnableParallel(context=retriever | format_docs, question=RunnablePassthrough())
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG chain created")

RAG chain created


In [24]:
# Query the chain
response = rag_chain.invoke("what is Okemakinde Sherif Sunday Education history?")
print(response)

I'm sorry, there is no specific information provided about Okemakinde Sherif Sunday's education history in the context you provided.


### Custom Prompts

In [25]:
# Create a custom prompt with specific instructions
custom_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a precise assistant. If you don't know the answer based on the context, say 'I don't know'."),
    ("human", "Context: {context}\n\nQuestion: {question}")
])

# Build chain with custom prompt
custom_rag = (
    RunnableParallel(context=retriever | format_docs, question=RunnablePassthrough())
    | custom_prompt
    | llm
    | StrOutputParser()
)

# Query the chain
response = rag_chain.invoke("Give me a complete list of all programming languages, frameworks, tools, and technologies mentioned in my CV.")
print(response)

response2 = rag_chain.invoke("What details from Okemakinde Sherif Sunday? Include name, phone, email, LinkedIn, GitHub, address")
print(response2)

response3 = rag_chain.invoke("Present my professional experience in chronological order with full details")
print(response3)



I'm sorry, but there is no mention of specific programming languages, frameworks, tools, or technologies in the provided context from your CV.
I'm sorry, there is no information provided about Okemakinde Sherif Sunday in the context you provided.
Based on the provided information, your professional experience in chronological order is as follows:

1. Nigerian Telecommunication Ltd., Abeokuta
   - Post: Cable Jointer/Cable Installer
   - 2004 – 2005

2. Mabisoft Automated Computer Ikeja, Lagos
   - Post: Events Planner
   - 2006

3. Link Serve Ltd, Lagos
   - Post: Vsat Installer, Dial-up/LAN Networking Engineer
   - 2006 – 2007

4. Federal Mortgage Bank of Nigeria
   - Post: I.T. Engineer
   - 2008 – 2009

5. INEC Ogun State
   - Post: Ad-Hoc Staff
   - 2010 – 2011

6. London Capital Ltd. Lagos Nigeria
   - Post: Marketing Consultant/IT Specialist
   - 2011

7. Dezeem Automobile Ibadan
   - 2012

8. Nigeria Security and Civil Defence Corps
   - Post: HOU ICT/Mobile CCTV Engineer (2019 

## Simple Conversational RAG

In [26]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import MessagesPlaceholder

# Store for chat histories
chat_store = {}

def get_session_history(session_id: str):
    if session_id not in chat_store:
        chat_store[session_id] = InMemoryChatMessageHistory()
    return chat_store[session_id]

# Create conversational prompt
conv_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer using the context provided."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "Context: {context}\n\nQuestion: {question}")
])

# Build base chain
conv_chain_base = (
    RunnableParallel(
        context=lambda x: format_docs(retriever.invoke(x["question"])),
        question=lambda x: x["question"],
        chat_history=lambda x: x.get("chat_history", [])
    )
    | conv_prompt
    | llm
    | StrOutputParser()
)

# Wrap with message history
conv_chain = RunnableWithMessageHistory(
    conv_chain_base,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

print("Conversational chain created")

Conversational chain created


In [27]:
# First question
response1 = conv_chain.invoke(
    {"question": "Present my professional experience in chronological order with full details."},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q1: Present my professional experience in chronological order with full details.")
print(f"A1: {response1}\n")

# Second question
response2 = conv_chain.invoke(
    {"question": "Group my experience by domain (e.g., web development, data science, management) and describe relevant roles."},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q2: Group my experience by domain (e.g., web development, data science, management) and describe relevant roles.")
print(f"A2: {response2}\n")

# Third question
response3 = conv_chain.invoke(
    {"question": "How many years of professional experience do I have according to the resume?"},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q3: How many years of professional experience do I have according to the resume?")
print(f"A3: {response3}\n")

# Fourth question
response4 = conv_chain.invoke(
    {"question": "Are there any employment gaps in my CV? If yes, mention the dates."},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q4: Are there any employment gaps in my CV? If yes, mention the dates.")
print(f"A4: {response4}\n")

# Fifth question
response5 = conv_chain.invoke(
    {"question": "What is the total duration of my experience in software engineering / [your field]?"},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q5: What is the total duration of my experience in software engineering / Statistics?")
print(f"A5: {response5}\n")

# Sixth question
response6 = conv_chain.invoke(
    {"question": "Extract and display the entire details of Okemakinde Sherif Sunday in a structured format: Name, Contact Info, Summary (if any), Work Experience (with bullets), Education, Skills, Projects, Certifications, and any other sections. Be as detailed and complete as possible."},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q6: Extract and display the entire details of Okemakinde Sherif Sunday in a structured format: Name, Contact Info, Summary, Work Experience, Education, Skills, Projects, Certifications, and other sections.")
print(f"A6: {response6}\n")

# Seventh question
response7 = conv_chain.invoke(
    {"question": "Provide a complete summary of my entire resume, including education, work experience, skills, and any other sections."},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q7: Provide a complete summary of my entire resume, including education, work experience, skills, and any other sections.")
print(f"A7: {response7}\n")

# Eighth question
response8 = conv_chain.invoke(
    {"question": "Summarize everything in my CV in detail, structured by section."},
    config={"configurable": {"session_id": "session1"}}
)
print(f"Q8: Summarize everything in my CV in detail, structured by section.")
print(f"A8: {response8}\n")


Q1: Present my professional experience in chronological order with full details.
A1: 1. Nigerian Telecommunication Ltd., Abeokuta (2004 – 2005)  
   - Post: Cable Jointer/Cable Installer  

2. Mabisoft Automated Computer Ikeja, Lagos (2006)  
   - Post: Events Planner  

3. Link Serve Ltd, Lagos (2006 – 2007)  
   - Post: Vsat Installer, Dial-up/LAN Networking Engineer  

4. Federal Mortgage Bank of Nigeria (2008 – 2009)  
   - Post: I.T. Engineer  

5. B.Sc. (Hons) Computer Engineering (2008)  

6. INEC Ogun State (2010 – 2011)  
   - Post: Ad-Hoc Staff  

7. London Capital Ltd. Lagos Nigeria (2011)  
   - Post: Marketing Consultant/IT Specialist  

8. Dezeem Automobile Ibadan (2012)  

9. Nigeria Security and Civil Defence Corps (2012 Till Date)  
   - Post: HOU ICT/Mobile CCTV Engineer (2019 Till Date)  
     - Radio/signal com. Instructor/operator  
     - Team leader enforcement of compliance on Standard & Regulations  

10. Certificate of Proficiency in IT (A+) (2010)  

11. Mast

In [28]:
# View chat history
session = get_session_history("session1")
print("Chat History:")
for msg in session.messages:
    print(f"\n{msg.type}: {msg.content}")

Chat History:

human: Present my professional experience in chronological order with full details.

ai: 1. Nigerian Telecommunication Ltd., Abeokuta (2004 – 2005)  
   - Post: Cable Jointer/Cable Installer  

2. Mabisoft Automated Computer Ikeja, Lagos (2006)  
   - Post: Events Planner  

3. Link Serve Ltd, Lagos (2006 – 2007)  
   - Post: Vsat Installer, Dial-up/LAN Networking Engineer  

4. Federal Mortgage Bank of Nigeria (2008 – 2009)  
   - Post: I.T. Engineer  

5. B.Sc. (Hons) Computer Engineering (2008)  

6. INEC Ogun State (2010 – 2011)  
   - Post: Ad-Hoc Staff  

7. London Capital Ltd. Lagos Nigeria (2011)  
   - Post: Marketing Consultant/IT Specialist  

8. Dezeem Automobile Ibadan (2012)  

9. Nigeria Security and Civil Defence Corps (2012 Till Date)  
   - Post: HOU ICT/Mobile CCTV Engineer (2019 Till Date)  
     - Radio/signal com. Instructor/operator  
     - Team leader enforcement of compliance on Standard & Regulations  

10. Certificate of Proficiency in IT (A+)

In [29]:
import chromadb
print(chromadb.__version__)

1.3.7
